In [1]:
import os
VERSION = 9
if VERSION is not None:
    os.environ["CC"] = f"/usr/bin/gcc-{VERSION}"
    os.environ["CXX"] = f"/usr/bin/g++-{VERSION}"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))

import torch
import matplotlib.pyplot as plt
import numpy as np
from utils import topo_utils

from gDel3D.build.gdel3d import Del
import plotly.graph_objects as go
import numpy as np
from utils.compare_quad import setup_camera
from utils.train_util import render
from data.camera import Camera
from models.ingp_color import pre_calc_cell_values, compute_vertex_colors_from_field, calculate_circumcenters_torch
from utils.graphics_utils import l2_normalize_th
import mediapy

torch.set_printoptions(precision=10)

tile_size = 16
height = 200
width = 200
fov = 60
viewmat = torch.eye(4)

viewmat, projection_matrix, cam_pos, fovy, fovx, fx, fy = setup_camera(height, width, fov, viewmat)
# Now extract R,T from viewmat
# If viewmat is truly "World->View", then R is top-left 3x3, T is top-right 3x1
# V = torch.inverse(viewmat)
V = viewmat
R = V[:3, :3].T
T = V[:3, 3]

# Create a blank image for the camera
blank_image = torch.zeros((3, height, width), device="cuda")

# Instantiate the camera
camera = Camera(
    colmap_id = 0,
    R = R.cpu().numpy(),
    T = T.cpu().numpy(),
    fovx = fovx,
    fovy = fovy,
    image = blank_image,
    gt_alpha_mask = None,
    uid = 0,
    cx = -1,
    cy = -1,
    trans = np.array([0.0, 0.0, 0.0]), # or any translation offset you need
    scale = 1.0,
    data_device = "cuda",
    # You can add any extra distortions, exposure, etc. you might need
)

vertices = torch.rand((200, 3), device='cuda')
velocity = l2_normalize_th(torch.rand((200, 3), device='cuda'))

frames = []

def field(xyz, scale=0.05):
    # Unpack coordinates
    x, y, z = xyz[..., 0], xyz[..., 1], xyz[..., 2]
    
    # Base color intercepts
    c0 = torch.sin(scale * x)
    c1 = torch.cos(scale * y)
    c2 = torch.sin(scale * z)
    
    # Gradients: 9 coefficients (to be reshaped as (3,3) per point if needed)
    c3  = torch.cos(scale * x)
    c4  = torch.sin(scale * (x + y))
    c5  = torch.cos(scale * (x + z))
    c6  = torch.sin(scale * (y - x))
    c7  = torch.cos(scale * y)
    c8  = torch.sin(scale * (y + z))
    c9  = torch.cos(scale * (z - x))
    c10 = torch.sin(scale * (z - y))
    c11 = torch.cos(scale * z)
    
    # Extra coefficient
    c12 = torch.sin(scale * (x * y * z))
    
    # Stack all coefficients along the last dimension
    return torch.stack([c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12], dim=-1)



/home/dronelab/delaunay_rasterization


In [ ]:
for i in range(200):
    print(i)
    vertices += velocity * 0.01
    model = lambda x: x
    model.vertices = vertices

    # Convert to tensor and move to CUDA
    v = Del(vertices.shape[0])
    indices_np, prev = v.compute(vertices.detach().cpu())
    indices_np = indices_np.numpy()
    indices_np = indices_np[(indices_np < vertices.shape[0]).all(axis=1)]
    indices = torch.as_tensor(indices_np).cuda()
    
    model.vertex_color = torch.ones((1), device='cuda')

    circumcenter, radius = calculate_circumcenters_torch(vertices[indices].double())
    output = field(circumcenter)
    density = (output[:, 0:1]-1).exp()
    field_samples = output[:, 1:]
    vcolors = compute_vertex_colors_from_field(
        vertices[indices].detach(), field_samples.float(), circumcenter.float().detach())
    vcolors = torch.nn.functional.softplus(vcolors, beta=10)
    vcolors = vcolors.reshape(-1, 12)
    features = torch.cat([
        density, vcolors], dim=1)
    model.tet_density = features.float()
    
    model.indices = indices
    model.scene_scaling = 1
    def get_cell_values(camera, mask=None):
        if mask is not None:
            return model.vertex_color, model.tet_density[mask]
        else:
            return model.vertex_color, model.tet_density
    model.get_cell_values = get_cell_values
    
    render_pkg = render(camera, model, tile_size=tile_size, min_t=0, ladder_p=1, pre_multi=1)
    torch_image = render_pkg['render'].permute(1, 2, 0)
    image = (torch_image.detach().cpu().numpy() * 255).clip(min=0, max=255).astype(np.uint8)
    frames.append(image)

mediapy.show_video(frames)


In [ ]:
vertices += velocity * 0.01
model = lambda x: x
model.vertices = vertices

# Convert to tensor and move to CUDA
v = Del(vertices.shape[0])
indices_np, prev = v.compute(vertices.detach().cpu())
indices_np = indices_np.numpy()
indices_np = indices_np[(indices_np < vertices.shape[0]).all(axis=1)]
indices = torch.as_tensor(indices_np).cuda()

model.vertex_color = torch.ones((1), device='cuda')

circumcenter, radius = calculate_circumcenters_torch(vertices[indices].double())
output = field(circumcenter)
density = (output[:, 0:1]-1).exp()
field_samples = output[:, 1:]
vcolors = compute_vertex_colors_from_field(
    vertices[indices].detach(), field_samples.float(), circumcenter.float().detach())
vcolors = torch.nn.functional.softplus(vcolors, beta=10)
vcolors = vcolors.reshape(-1, 12)
features = torch.cat([
    density, vcolors], dim=1)
model.tet_density = features.float()

model.indices = indices
model.scene_scaling = 1
def get_cell_values(camera, mask=None):
    if mask is not None:
        return model.vertex_color, model.tet_density[mask]
    else:
        return model.vertex_color, model.tet_density
model.get_cell_values = get_cell_values



In [ ]:

render_pkg = render(camera, model, tile_size=tile_size, min_t=0, ladder_p=1, pre_multi=1)
torch_image = render_pkg['render'].permute(1, 2, 0)
image = (torch_image.detach().cpu().numpy() * 255).clip(min=0, max=255).astype(np.uint8)
frames.append(image)

mediapy.show_video(frames)

"<source src=""data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAABC1tZGF0AAACfwYF//973EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2NCByMzA5NSBiYWVlNDAwIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMiAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTYgbG9va2FoZWFkX3RocmVhZHM9MSBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxhY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHlyYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3ZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTI1IHNjZW5lY3V0PTQwIGludHJhX3JlZnJlc2g9MCByYz1jcXAgbWJ0cmVlPTAgcXA9MjAgaXBfcmF0aW89MS40MCBwYl9yYXRpbz0xLjMwIGFxPTAAgAAAAUdliIQA//70oPgU0GJjysxWvdiImDlA4OycPwQPczVsBStsObkAAHWANIxuptCQGibKc7dE9/f7IxKcFUvbxjClxg0ifuLfJDP4QtWIjiV7+392Is8Mkt5Gu1/6UBxznQEIsaWiw+774+FsPSCpC+uZlKUro4kJYv/16LgBkMz9LZB1lDAeHxXrDzVF9hh91QWLZShAW5xRjkXsnJ5iZxCZTR5lK5IQ305gN2K1r/2eOW0zSQ34j9WP7Tx0ArADDEdPnI+Bs+5DEyvDjz4POHDQqrPWi4xFu4XTQ64sNTZHkvPqHrq4NqHo/p2QGeNTYcfukdxQSDXcW3YcvpfNVCHaxdz3Cy2O+KhnMYbqsINx5XK/pn6VQe+jiRmljgBo8ZUyKUtBrWVJssh6AezDoEIvxp7IQdVWvRAW1j18oKJbEgYIemJv8R0AAABTQZohbH/kQAfLjh5jFkGjbT/kHlgklzyaIOyAbXEuUPt69j5Aky3PvsKFWHx4p2FhDgPlfHFoZheoelrS8pAu1Q22TPm7batHPT3G0t/Rg3JS3uIAAAMjbW9vdgAAAGxtdmhkAAAAAAAAAAAAAAAAAAAD6AAAACIAAQAAAQAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAk10cmFrAAAAXHRraGQAAAADAAAAAAAAAAAAAAABAAAAAAAAACIAAAAAAAAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAMgAAADIAAAAAAAkZWR0cwAAABxlbHN0AAAAAAAAAAEAAAAiAAAAAAABAAAAAAHFbWRpYQAAACBtZGhkAAAAAAAAAAAAAAAAAAA8AAAAAgBVxAAAAAAALWhkbHIAAAAAAAAAAHZpZGUAAAAAAAAAAAAAAABWaWRlb0hhbmRsZXIAAAABcG1pbmYAAAAUdm1oZAAAAAEAAAAAAAAAAAAAACRkaW5mAAAAHGRyZWYAAAAAAAAAAQAAAAx1cmwgAAAAAQAAATBzdGJsAAAAsHN0c2QAAAAAAAAAAQAAAKBhdmMxAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAMgAyABIAAAASAAAAAAAAAABFUxhdmM2MC4zMS4xMDIgbGlieDI2NAAAAAAAAAAAAAAAGP//AAAANmF2Y0MBZAAN/+EAGWdkAA2s2UNG+WWEAAADAAQAAAMB4DxQplgBAAZo6+GyyLD9+PgAAAAAFGJ0cnQAAAAAAAPisAAD4rAAAAAYc3R0cwAAAAAAAAABAAAAAgAAAQAAAAAUc3RzcwAAAAAAAAABAAAAAQAAABxzdHNjAAAAAAAAAAEAAAABAAAAAgAAAAEAAAAcc3RzegAAAAAAAAAAAAAAAgAAA84AAABXAAAAFHN0Y28AAAAAAAAAAQAAADAAAABidWR0YQAAAFptZXRhAAAAAAAAACFoZGxyAAAAAAAAAABtZGlyYXBwbAAAAAAAAAAAAAAAAC1pbHN0AAAAJal0b28AAAAdZGF0YQAAAAEAAAAATGF2ZjYwLjE2LjEwMA=="" type=""video/mp4""/> This browser does not support the video tag."
